In [ ]:
# 1942年McCulloch Pitts 神经元模型
# f（ΣXi*Wi+b）————f-激活函数activation function、b-偏置项bias
# 常用的激活函数：
#     ReLU f(x)=max(x,0)→tf.nn.relu()
#     Sigmoid f(x)=1/(1+1/e^x)→tf.nn.sigmoid()         limf(+∞)=1，limf(-∞)=0，f(0)=0.5
#     TanH f(x)=(1-1/(e^2))/(1+1/(e^2))→tf.nn.tanh()   limf(+∞)=1，limf(-∞)=-1，f(0)=0
#     Swish f(x)=x*Sigmoid                              limf(+∞)=+∞，limf(-∞)=-0，f(0)=0
# NN优化的目标：loss最小   均方误差MSE（Mean Squared Error），mse(y_,y)=(Σ(y-y_)^2)/n→loss_mse=tf.reduce_mean(tf.square(y_-y))
#                          自定义,loss(y_,y)=Σf(y_,y)
#                          交叉熵CE（Cross Entropy）,表征两个概率分布之间的距离,概率分布越远，交叉熵分布越小
#                                                    H(y_,y)=-Σ(y_*logy)→ce=tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-12,1.0)))

In [ ]:
# softmax：某元素的指数与所有元素指数和的比值
# softmax用于多分类过程中，它将多个神经元的输出，映射到（0,1）区间内，可以看成概率来理解，从而来进行多分类
# softmax(Yi)=(e^Yi)/(Σe^Yi)→ce=tf.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
#                             cem=tf.reduce_mean(ce)     当前预测值与标准答案的差距=loss

In [17]:
#导入模块，生成模拟数据集
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
BATCH_SIZE = 4   #一次喂入神经网络8组数据
seed = 23455     #随机种子
COST = 9
PROFIT = 1

In [18]:
rdm = np.random.RandomState(seed)
X = rdm.rand(32,2)
Y_ = [[x1+x2+(rdm.rand()/10.0-0.05)] for (x1,x2) in X]

In [19]:
#定义NN输入、参数、输出,定义前向传播
x = tf.placeholder(tf.float32,shape=(None,2))
y_ = tf.placeholder(tf.float32,shape=(None,1))
w1 = tf.Variable(tf.random.normal([2,1],stddev=1,seed=1))
y = tf.matmul(x,w1)

In [20]:
#定义损失函数及反向传播方法
loss = tf.reduce_sum(tf.where(tf.greater(y,y_),(y-y_)*COST,(y_-y)*PROFIT))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)
#train_step = tf.train.MomentumOptimizer(0.001).minimize(loss)
#train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [22]:
#生成会话，训练steps轮
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
   
    #训练模型
    STEPS = 3000
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        sess.run(train_step,feed_dict={x:X[start:end],y_: Y_[start:end]})
        if i % 500 == 0:
            total_loss = sess.run(loss,feed_dict={x: X,y_ : Y_})
            print("After %d traning step(s),loss on all data is %g"%(i,total_loss))
            print("w1:\n",sess.run(w1))
        
    #输出训练后的参数值
    print(" final w1:\n",sess.run(w1))


After 0 traning step(s),loss on all data is 29.8645
w1:
 [[-0.8088941]
 [ 1.4859729]]
After 500 traning step(s),loss on all data is 14.341
w1:
 [[0.02260804]
 [1.1564052 ]]
After 1000 traning step(s),loss on all data is 2.39831
w1:
 [[0.8723926]
 [1.0072129]]
After 1500 traning step(s),loss on all data is 1.47386
w1:
 [[0.95760536]
 [0.97100633]]
After 2000 traning step(s),loss on all data is 1.4655
w1:
 [[0.96072936]
 [0.9691097 ]]
After 2500 traning step(s),loss on all data is 1.45911
w1:
 [[0.9603584]
 [0.9706179]]
 final w1:
 [[0.9613664]
 [0.9762951]]
